In [3]:
import os
import pandas as pd
import numpy as np
import faiss

# === Configuration des chemins ===
DATA_PATH = os.path.join("..", "scripts", "data", "processed")
DATA_FILE = os.path.join(DATA_PATH, "events_with_embeddings.csv")
INDEX_DIR = os.path.join(DATA_PATH, "index")

# Création automatique du dossier d’index
os.makedirs(INDEX_DIR, exist_ok=True)
print(f"Dossier d'index vérifié/créé : {INDEX_DIR}")

# === Lecture du fichier CSV ===
df = pd.read_csv(DATA_FILE)
print(f"✅ Fichier chargé : {DATA_FILE}")
print(f"Nombre de lignes : {len(df)}\n")

# === Fonction de conversion string → array ===
def str_to_list(emb_str):
    """Convertit une chaîne d'embedding en liste numérique"""
    if pd.isna(emb_str):
        return None
    try:
        return np.array(eval(emb_str), dtype='float32')
    except Exception as e:
        print(f"⚠️ Erreur conversion embedding : {e}")
        return None


# === Fonction d’indexation générique ===
def create_faiss_index(df, column_name, output_filename):
    """Crée et sauvegarde un index FAISS pour une colonne d’embeddings donnée"""
    print(f"\n🧠 Création de l'index pour : {column_name}")

    # Conversion et nettoyage
    embeddings_list = df[column_name].dropna().apply(str_to_list).dropna().to_list()

    if len(embeddings_list) == 0:
        print(f"❌ Aucune donnée valide dans {column_name}, index non créé.")
        return

    embeddings_array = np.vstack(embeddings_list)
    print(f"Shape embeddings ({column_name}) : {embeddings_array.shape}")

    # Création de l’index
    embedding_dim = embeddings_array.shape[1]
    index = faiss.IndexFlatL2(embedding_dim)
    index.add(embeddings_array)

    # Sauvegarde
    output_path = os.path.join(INDEX_DIR, output_filename)
    faiss.write_index(index, output_path)
    print(f"💾 Index FAISS sauvegardé dans : {output_path}")
    print(f"✅ {index.ntotal} vecteurs indexés avec succès.")


# === Création des deux index ===
create_faiss_index(df, "embedding_description", "faiss_index_description.index")
create_faiss_index(df, "embedding_description_longue", "faiss_index_description_longue.index")

print("\n🎯 Tous les index FAISS ont été créés avec succès !")


Dossier d'index vérifié/créé : ..\scripts\data\processed\index
✅ Fichier chargé : ..\scripts\data\processed\events_with_embeddings.csv
Nombre de lignes : 9973


🧠 Création de l'index pour : embedding_description
Shape embeddings (embedding_description) : (9973, 1024)
💾 Index FAISS sauvegardé dans : ..\scripts\data\processed\index\faiss_index_description.index
✅ 9973 vecteurs indexés avec succès.

🧠 Création de l'index pour : embedding_description_longue
Shape embeddings (embedding_description_longue) : (9973, 1024)
💾 Index FAISS sauvegardé dans : ..\scripts\data\processed\index\faiss_index_description_longue.index
✅ 9973 vecteurs indexés avec succès.

🎯 Tous les index FAISS ont été créés avec succès !
